# **Важно!** 

Домашнее задание состоит из нескольких задач, которые вам нужно решить.
*   Баллы выставляются по принципу выполнено/невыполнено.
*   За каждую выполненую задачу вы получаете баллы (количество баллов за задание указано в скобках).

**Инструкция выполнения:** Выполните задания в этом же ноутбуке (места под решения **КАЖДОЙ** задачи обозначены как **#НАЧАЛО ВАШЕГО РЕШЕНИЯ** и **#КОНЕЦ ВАШЕГО РЕШЕНИЯ**)

**Как отправить задание на проверку:** Вам необходимо сохранить ваше решение в данном блокноте и отправить итоговый **файл .IPYNB** в личном сообщении Telegram.

# **Прежде чем проверять задания:**

1. Перезапустите **ядро (restart the kernel)**: в меню, выбрать **Ядро (Kernel)**
→ **Перезапустить (Restart)**
2. Затем **Выполнить** **все ячейки (run all cells)**: в меню, выбрать **Ячейка (Cell)**
→ **Запустить все (Run All)**.

---

## Введение в Parameter Efficient Fine-Tuning (PEFT)

В данном задании основное внимание уделяется изучению и применению современных методов **Parameter Efficient Fine-Tuning (PEFT)**. Эти подходы позволяют эффективно дообучать большие языковые модели, используя лишь небольшую часть параметров, что критически важно при работе с ограниченными вычислительными ресурсами.

### Обязательные PEFT методы для изучения:

1. **LoRA (Low-Rank Adaptation)**
   - Разложение весовых матриц на произведение матриц низкого ранга
   - Ключевые гиперпараметры: `r` (rank), `alpha`, `dropout`, `target_modules`

2. **QLoRA (Quantized LoRA)** 
   - Комбинация 4-bit квантизации (NF4) с LoRA
   - Значительно снижает потребление GPU памяти

3. **AdaLoRA (Adaptive LoRA)**
   - Динамическое изменение ранга во время обучения
   - Автоматическая оптимизация распределения параметров

### Дополнительные методы (по выбору):
- **IA³ (Infused Adapter by Inhibiting and Amplifying Inner Activations)**
- **Prefix Tuning / P-Tuning v2**
- **Prompt Tuning**

### Методология сравнения PEFT подходов:

Для каждого метода необходимо измерить и сравнить:

#### Эффективность ресурсов:
- **Количество обучаемых параметров** (в % от общего числа параметров модели)
- **Потребление GPU памяти** (в GB во время обучения и инференса)
- **Время обучения** (сек/эпоху)
- **Скорость инференса** (токенов/секунду)

#### Качество результатов:
- **Основные метрики** в зависимости от задачи (ROUGE для суммаризации, BLEU для перевода)
- **Стабильность обучения** (сходимость функции потерь)
- **Качественный анализ** выходных текстов

#### Требования к отчету:
1. **Сравнительная таблица** всех протестированных методов
2. **Графики Pareto-frontier**: эффективность vs качество
3. **Обоснованные рекомендации** по выбору метода для различных сценариев
4. **Анализ компромиссов** между точностью и эффективностью

---

### Задание 1: Parameter Efficient Fine-Tuning (PEFT) моделей для суммаризации текстов

Цель задания — научиться применять современные методы параметрически-эффективного дообучения (PEFT) для настройки больших языковых моделей на задачу суммаризации текстов. Вы реализуете различные PEFT подходы: LoRA, QLoRA из библиотеки PEFT.

#### Задачи:

1. **Выбор датасета**:
   - Загрузите датасет для задачи суммаризации, например, датасет `CNN/DailyMail`, содержащий новостные статьи и их краткие содержания (референсы). Используйте библиотеку `datasets` для загрузки данных.
   - Обратите внимание на то, что вы можете использовать и другие подходящие датасеты для суммаризации (например, XSum).

2. **Предобработка данных**:
   - **Разделите** данные на обучающую и тестовую выборки.
   - **Очистите текст** от лишних символов, специальных токенов и пробелов.
   - **Подготовьте данные** в формате, подходящем для выбранной модели:
     - Для GPT-2 вам нужно будет подать текст целиком (входной текст + референсное суммирование в одном формате).
     - Для T5 модель требует форматировать входные данные в виде `summarize: <текст>` для текстов, которые нужно суммировать.

3. **Создание модели**:
   - **GPT-2**:
     - Импортируйте предобученную модель `GPT2LMHeadModel` из библиотеки Hugging Face.
     - Модель GPT-2 изначально не предобучена для задачи суммаризации, поэтому требуется её дообучение на подходящих данных.
     - Поддержите правильное управление длиной сгенерированного текста при инференсе, чтобы обеспечить краткость суммаризаций.
   
   - **T5**:
     - Для T5 используйте модель `T5ForConditionalGeneration`. T5 уже предобучена на множестве задач, включая суммаризацию, поэтому она лучше подходит для данной задачи.
     - В отличие от GPT-2, модель T5 обучена на задаче, где входной текст — это задание (например, "summarize:") + текст для обработки, а выход — краткое содержание. Это нужно учесть при подготовке данных".
     - Импортируйте модель из библиотеки Hugging Face и настройте её для использования на задаче суммаризации.

4. **Настройка параметров обучения**:
   - Настройте параметры обучения для обеих моделей:
     - Количество эпох.
     - Размер батча.
     - Скорость обучения.
     - Выберите оптимизатор (например, AdamW).
   - Для T5 используйте кросс-энтропийную функцию потерь (`CrossEntropyLoss`), так как это задача генерации текста с "условной вероятностью". Для GPT-2 используйте ту же функцию с учётом автогрегрессивного генерационного процесса.

5. **PEFT Fine-tuning модели** (основная часть задания):
   - **Обязательно** реализуйте дообучение с использованием различных PEFT методов:
     - **LoRA (Low-Rank Adaptation)**: Настройте параметры rank (r), alpha, dropout
     - **QLoRA (Quantized LoRA)**: Используйте 4-bit квантизацию с LoRA
     - **AdaLoRA**: Адаптивное изменение ранга во время обучения
     - **Дополнительно**: попробуйте IA³ (Infused Adapter by Inhibiting and Amplifying Inner Activations) или Prompt Tuning
   - Для каждого PEFT метода:
     - Настройте специфические гиперпараметры (rank, alpha, target_modules)
     - Измерьте количество обучаемых параметров
     - Зафиксируйте время обучения и потребление памяти
   - Используйте класс `Trainer` или `SFTTrainer` из библиотеки `trl` для процесса дообучения
   - Сравните результаты full fine-tuning с PEFT подходами (опционально)

6. **Инференс**:
   - Используйте обе модели для генерации кратких содержаний на тестовой выборке.
   - Подготовьте несколько примеров суммаризаций и выведите результаты для каждой модели.
   - Для инференса используйте разные стратегии декодирования:
     - **Greedy decoding** (жадный поиск).
     - **Beam search** (поиск по нескольким лучам).
     - **Sampling** (стохастическая генерация с использованием вероятностей).
   - Сравните результаты, чтобы понять, как разные стратегии влияют на качество суммаризаций.

7. **Сравнительная оценка PEFT методов**:
   - Создайте сравнительную таблицу для всех протестированных методов:
     - Количество обучаемых параметров (в % от общего числа параметров модели)
     - Время обучения на эпоху
     - Потребление GPU памяти
     - Качество суммаризации по метрикам ROUGE и BLEU
   - Оцените качество суммаризаций с использованием метрик:
     - **ROUGE-1, ROUGE-2, ROUGE-L** — для оценки точности и полноты суммаризаций
     - **BLEU** — для оценки схожести с референсным текстом
   - Проанализируйте trade-off между эффективностью обучения и качеством результата

#### Ожидаемые результаты:
- **Код с реализацией различных PEFT методов** для выбранной модели (GPT-2 или T5)
- **Сравнительный анализ** всех протестированных PEFT подходов с детальными метриками эффективности
- **Отчет** с обоснованием выбора оптимального PEFT метода для задачи суммаризации
- **Примеры суммаризаций** для каждого PEFT метода с качественным анализом различий
- **Рекомендации** по выбору PEFT подхода в зависимости от ограничений по ресурсам

#### Рекомендуемые ресурсы:
- **[PEFT Documentation](https://huggingface.co/docs/peft/index)** - основная документация библиотеки PEFT
- **[LoRA Developer Guide](https://huggingface.co/docs/peft/developer_guides/lora)** - детальное руководство по LoRA
- **[QLoRA Implementation](https://huggingface.co/docs/peft/developer_guides/quantization)** - квантизация и QLoRA
- [Документация Hugging Face Transformers](https://huggingface.co/docs/transformers/index)
- [Документация Hugging Face Datasets](https://huggingface.co/docs/datasets/index)
- **[PEFT Examples](https://github.com/huggingface/peft/tree/main/examples)** - примеры использования различных PEFT методов
- **[TRL SFTTrainer](https://huggingface.co/docs/trl/sft_trainer)** - для supervised fine-tuning

#### Критерии оценки:
- **Корректность реализации PEFT методов** (30%) - правильная настройка и применение различных PEFT подходов
- **Полнота сравнительного анализа** (25%) - детальное сравнение методов по всем указанным метрикам
- **Качество сгенерированных суммаризаций** (20%) - оценка по ROUGE/BLEU метрикам
- **Глубина анализа и выводов** (15%) - обоснованные рекомендации по выбору PEFT метода
- **Четкость и структурированность отчета** (10%) - качество документации процесса и результатов

In [1]:
# %%
# 1) Imports & basic setup
import os
import time
from typing import Dict, Any, List, Optional

import torch
from datasets import load_dataset
import evaluate

from transformers import (
    AutoTokenizer,
    T5ForConditionalGeneration,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    BitsAndBytesConfig,
)

from peft import LoraConfig, AdaLoraConfig, get_peft_model, prepare_model_for_kbit_training

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using: {DEVICE}")
if DEVICE.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(torch.cuda.current_device()))


/home/vladislav/miniconda3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using: cuda
GPU: NVIDIA GeForce RTX 3080 Laptop GPU


In [2]:

# %%
# 2) Small utility helpers

def count_trainable(model: torch.nn.Module) -> Dict[str, Any]:
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return {
        "total": int(total),
        "trainable": int(trainable),
        "trainable_pct": 100.0 * trainable / total,
    }


def cuda_mem_stats() -> Dict[str, int]:
    if not torch.cuda.is_available():
        return {"allocated_bytes": 0, "reserved_bytes": 0}
    idx = torch.cuda.current_device()
    return {
        "allocated_bytes": int(torch.cuda.memory_allocated(idx)),
        "max_allocated_bytes": int(torch.cuda.max_memory_allocated(idx)),
        "reserved_bytes": int(torch.cuda.memory_reserved(idx)),
        "max_reserved_bytes": int(torch.cuda.max_memory_reserved(idx)),
    }


In [3]:

# %%
# 3) Data: load & preprocess (CNN/DailyMail)

def load_cnn_dm(sample_train: Optional[int] = None, dataset_config: str = "3.0.0"):
    ds = load_dataset("cnn_dailymail", dataset_config)
    train = ds["train"]
    valid = ds["validation"] if "validation" in ds else ds["test"]
    if sample_train:
        train = train.select(range(min(sample_train, len(train))))
        valid_n = max(1, sample_train // 10)
        valid = valid.select(range(min(valid_n, len(valid))))
    return train, valid


def _strip_fields(example: Dict[str, Any]) -> Dict[str, Any]:
    for k in ("article", "highlights", "summary", "document", "text"):
        if k in example and isinstance(example[k], str):
            example[k] = example[k].strip()
    return example


def preprocess_for_t5(tokenizer: AutoTokenizer, examples: Dict[str, List[str]],
                      max_input: int = 512, max_target: int = 128) -> Dict[str, Any]:
    inputs = ["summarize: " + art for art in examples["article"]]
    model_inputs = tokenizer(
        inputs,
        max_length=max_input,
        truncation=True,
        padding="max_length",
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["highlights"],
            max_length=max_target,
            truncation=True,
            padding="max_length",
        )["input_ids"]
    labels = [[-100 if tok == tokenizer.pad_token_id else tok for tok in seq] for seq in labels]
    model_inputs["labels"] = labels
    return model_inputs


In [4]:

# %%
# 4) Unified PEFT model factory (T5): LoRA / AdaLoRA / QLoRA

def create_t5_with_peft(
    model_name: str = "t5-base",
    target_modules: Optional[List[str]] = None,
    mode: str = "lora",  # "lora", "adalora", "qlora"
    lora_r: int = 16,
    lora_alpha: int = 32,
    lora_dropout: float = 0.05,
    # AdaLoRA
    adalora_init_r: int = 4,
    adalora_tinit: int = 100,
    adalora_tfinal: int = 1000,
    total_steps: Optional[int] = None,
):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token or "<pad>"

    tmods = target_modules or ["q", "k", "v", "o", "wi_0", "wi_1", "wo"]

    if mode.lower() == "qlora":
        compute_dtype = (
            torch.bfloat16
            if (torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8)
            else torch.float16
        )
        bnb_cfg = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=compute_dtype,
        )
        model = T5ForConditionalGeneration.from_pretrained(
            model_name, quantization_config=bnb_cfg, device_map="auto",
        )
        model = prepare_model_for_kbit_training(model)
        peft_cfg = LoraConfig(
            r=lora_r, lora_alpha=lora_alpha, lora_dropout=lora_dropout,
            bias="none", task_type="SEQ_2_SEQ_LM", target_modules=tmods,
        )
        quant_desc = "4-bit (QLoRA)"
    else:
        torch_dtype = (
            torch.bfloat16
            if (torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8)
            else torch.float16
        )
        model = T5ForConditionalGeneration.from_pretrained(
            model_name, torch_dtype=torch_dtype, device_map="auto",
        )
        if mode.lower() == "adalora":
            steps = int(total_steps or 1)

            # derive a safe schedule from total_steps
            tinit  = max(1, int(0.05 * steps))             # ~5% warmup
            tfinal = max(tinit + 10, int(0.30 * steps))    # ~30% end of budgeting
            if tfinal >= steps:
                tfinal = max(tinit + 1, steps - 1)

            # (optional) fallback for ultra-tiny runs
            if steps < 20:
                print(f"[AdaLoRA] total_step={steps} too small; switching to plain LoRA.")
                peft_cfg = LoraConfig(
                    r=lora_r, lora_alpha=lora_alpha, lora_dropout=lora_dropout,
                    bias="none", task_type="SEQ_2_SEQ_LM", target_modules=tmods
                )
            else:
                peft_cfg = AdaLoraConfig(
                    r=lora_r,
                    lora_alpha=lora_alpha,
                    target_modules=tmods,
                    init_r=adalora_init_r,
                    tinit=tinit,
                    tfinal=tfinal,
                    total_step=steps,
                    lora_dropout=lora_dropout,
                    bias="none",
                    task_type="SEQ_2_SEQ_LM",
                )
            print(f"[AdaLoRA] steps={steps}, tinit={tinit}, tfinal={tfinal}")
            quant_desc = "fp/bf16 (LoRA)"
        else:
            peft_cfg = LoraConfig(
                r=lora_r, lora_alpha=lora_alpha, lora_dropout=lora_dropout,
                bias="none", task_type="SEQ_2_SEQ_LM", target_modules=tmods,
            )
            quant_desc = "fp/bf16 (LoRA)"

    # training-friendly flags
    model.config.use_cache = False
    if hasattr(model, "enable_input_require_grads"):
        model.enable_input_require_grads()

    model = get_peft_model(model, peft_cfg)

    # safety guard
    trainables = sum(p.numel() for p in model.parameters() if p.requires_grad)
    if trainables == 0:
        raise RuntimeError(f"Adapters not attached — check target_modules={tmods}")

    print(f"Mode: {mode} | Base: {quant_desc}")
    model.print_trainable_parameters()
    return tokenizer, model


In [5]:

# %%
# 5) Training arguments & Trainer builder

def make_training_args(
    out_dir: str = "./out/t5_peft",
    train_bs: int = 4,
    eval_bs: int = 4,
    epochs: int = 2,
    lr: float = 2e-4,
    logging_steps: int = 50,
) -> TrainingArguments:
    return TrainingArguments(
        output_dir=out_dir,
        per_device_train_batch_size=train_bs,
        per_device_eval_batch_size=eval_bs,
        num_train_epochs=epochs,
        learning_rate=lr,
        weight_decay=0.01,
        logging_steps=logging_steps,
        eval_strategy="epoch",
        save_strategy="epoch",
        fp16=(torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] < 8),
        bf16=(torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8),
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        lr_scheduler_type="cosine",
        warmup_ratio=0.03,
        report_to=["none"],
    )


def build_trainer(model, tokenizer, train_ds, val_ds, args: TrainingArguments) -> Trainer:
    collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=-100, padding=True)
    return Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        data_collator=collator,
    )


In [6]:

# %%
# 6) Metrics (ROUGE + BLEU)

_rouge = evaluate.load("rouge")
_bleu = evaluate.load("bleu")


def compute_text_metrics(preds: List[str], refs: List[str]) -> Dict[str, Any]:
    r = _rouge.compute(predictions=preds, references=refs)
    b = _bleu.compute(predictions=preds, references=refs)
    return {
        "rouge1": r.get("rouge1"),
        "rouge2": r.get("rouge2"),
        "rougeL": r.get("rougeL"),
        "bleu": b.get("bleu"),
    }


In [7]:

# %%
# 7) End-to-end: run PEFT on T5 (mode selectable)

def run_t5_peft(
    model_name: str = "t5-base",
    sample_train: int = 2000,
    out_dir: str = "./out/t5_peft",
    epochs: int = 2,
    train_bs: int = 4,
    eval_bs: int = 4,
    mode: str = "lora",  # "lora", "adalora", "qlora"
) -> Dict[str, Any]:
    # data
    train_raw, val_raw = load_cnn_dm(sample_train=sample_train)
    train_raw = train_raw.filter(lambda x: (x.get("highlights") or "").strip() != "")
    val_raw = val_raw.filter(lambda x: (x.get("highlights") or "").strip() != "")
    train_raw = train_raw.map(_strip_fields)
    val_raw = val_raw.map(_strip_fields)

    # model (estimate total steps for AdaLoRA scheduling)
    est_steps = max(1, (len(train_raw) // max(1, train_bs)) * max(1, epochs))
    tokenizer, model = create_t5_with_peft(
        model_name=model_name,
        mode=mode,
        total_steps=est_steps,
    )

    # tokenize
    train_tok = train_raw.map(lambda ex: preprocess_for_t5(tokenizer, ex), batched=True, remove_columns=train_raw.column_names)
    val_tok = val_raw.map(lambda ex: preprocess_for_t5(tokenizer, ex), batched=True, remove_columns=val_raw.column_names)

    # trainer
    args = make_training_args(out_dir=out_dir, train_bs=train_bs, eval_bs=eval_bs, epochs=epochs)
    trainer = build_trainer(model, tokenizer, train_tok, val_tok, args)

    # quick sanity
    sample = train_tok.select(range(min(4, len(train_tok))))
    batch = trainer.data_collator(sample)
    print("Batch keys:", list(batch.keys()))

    # train
    t0 = time.time()
    trainer.train()
    train_time = time.time() - t0

    # generate on a few examples
    model.to(DEVICE)
    model.eval()
    few = val_raw.select(range(min(5, len(val_raw))))
    prompts = ["summarize: " + s["article"] for s in few]
    enc = tokenizer(prompts, return_tensors="pt", truncation=True, padding=True, max_length=512).to(DEVICE)

    gen_beam = model.generate(**enc, max_new_tokens=128, num_beams=4)
    gen_greedy = model.generate(**enc, max_new_tokens=128)
    gen_sample = model.generate(**enc, max_new_tokens=128, do_sample=True, top_k=50, top_p=0.95)

    pred_beam = tokenizer.batch_decode(gen_beam, skip_special_tokens=True)
    pred_greedy = tokenizer.batch_decode(gen_greedy, skip_special_tokens=True)
    pred_sample = tokenizer.batch_decode(gen_sample, skip_special_tokens=True)

    refs = [s["highlights"] for s in few]

    m_beam = compute_text_metrics(pred_beam, refs)
    m_greedy = compute_text_metrics(pred_greedy, refs)
    m_sample = compute_text_metrics(pred_sample, refs)

    return {
        "trainable_params": count_trainable(model),
        "gpu_mem": cuda_mem_stats(),
        "train_time_s": train_time,
        "metrics": {
            "beam": m_beam,
            "greedy": m_greedy,
            "sample": m_sample,
        },
        "examples": {
            "inputs": prompts,
            "beam": pred_beam,
            "greedy": pred_greedy,
            "sample": pred_sample,
        },
    }


In [8]:
CONFIG = {
    "model_name": "t5-base",      
    "sample_train": 2000,         
    "out_dir": "./out/t5_lora",
    "epochs": 2,
    "train_bs": 4,
    "eval_bs": 4,
    "mode": "lora",            
}

results = run_t5_peft(
    model_name=CONFIG["model_name"],
    sample_train=CONFIG["sample_train"],
    out_dir=CONFIG["out_dir"],
    epochs=CONFIG["epochs"],
    train_bs=CONFIG["train_bs"],
    eval_bs=CONFIG["eval_bs"],
    mode=CONFIG["mode"],
)
print("Training done.\n")

print("Trainable parameters:", results["trainable_params"])
print("GPU memory:", results["gpu_mem"]) 
print("Training time (s):", round(results["train_time_s"], 2))
print("\nMetrics (beam):", results["metrics"]["beam"]) 
print("Metrics (greedy):", results["metrics"]["greedy"]) 
print("Metrics (sample):", results["metrics"]["sample"]) 

print("\nExamples:")
for i, src in enumerate(results["examples"]["inputs"]):
    print("-" * 80)
    print("INPUT:", src[:200].replace("\n", " ") + ("..." if len(src) > 200 else ""))
    print("BEAM:", results["examples"]["beam"][i])
    print("GREEDY:", results["examples"]["greedy"][i])
    print("SAMPLE:", results["examples"]["sample"][i])


`torch_dtype` is deprecated! Use `dtype` instead!
/tmp/ipykernel_16701/3814343219.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  return Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


Mode: lora | Base: fp/bf16 (LoRA)
trainable params: 5,013,504 || all params: 227,917,056 || trainable%: 2.1997
Batch keys: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids']


Epoch,Training Loss,Validation Loss
1,1.635500,1.779909
2,1.553000,1.777216


Training done.

Trainable parameters: {'total': 227917056, 'trainable': 5013504, 'trainable_pct': 2.199705492861403}
GPU memory: {'allocated_bytes': 524061184, 'max_allocated_bytes': 1012930560, 'reserved_bytes': 1130364928, 'max_reserved_bytes': 1130364928}
Training time (s): 310.44

Metrics (beam): {'rouge1': 0.2595959595959596, 'rouge2': 0.0673992673992674, 'rougeL': 0.19272727272727272, 'bleu': 0.05758046547802751}
Metrics (greedy): {'rouge1': 0.28637765896288864, 'rouge2': 0.08303310051323051, 'rougeL': 0.2104155518674534, 'bleu': 0.07195675442095749}
Metrics (sample): {'rouge1': 0.2482261966400138, 'rouge2': 0.0731620151434393, 'rougeL': 0.1728313358695596, 'bleu': 0.0}

Examples:
--------------------------------------------------------------------------------
INPUT: summarize: (CNN)Share, and your gift will be multiplied. That may sound like an esoteric adage, but when Zully Broussard selflessly decided to give one of her kidneys to a stranger, her generosity pai...
BEAM: Zully 

In [9]:

CONFIG = {
    "model_name": "t5-base",     
    "sample_train": 2000,        
    "out_dir": "./out/t5_adalora",
    "epochs": 2,
    "train_bs": 4,
    "eval_bs": 4,
    "mode": "adalora",         
}

results = run_t5_peft(
    model_name=CONFIG["model_name"],
    sample_train=CONFIG["sample_train"],
    out_dir=CONFIG["out_dir"],
    epochs=CONFIG["epochs"],
    train_bs=CONFIG["train_bs"],
    eval_bs=CONFIG["eval_bs"],
    mode=CONFIG["mode"],
)
print("Training done.\n")

print("Trainable parameters:", results["trainable_params"])
print("GPU memory:", results["gpu_mem"]) 
print("Training time (s):", round(results["train_time_s"], 2))
print("\nMetrics (beam):", results["metrics"]["beam"]) 
print("Metrics (greedy):", results["metrics"]["greedy"]) 
print("Metrics (sample):", results["metrics"]["sample"]) 

print("\nExamples:")
for i, src in enumerate(results["examples"]["inputs"]):
    print("-" * 80)
    print("INPUT:", src[:200].replace("\n", " ") + ("..." if len(src) > 200 else ""))
    print("BEAM:", results["examples"]["beam"][i])
    print("GREEDY:", results["examples"]["greedy"][i])
    print("SAMPLE:", results["examples"]["sample"][i])


/home/vladislav/miniconda3/envs/nlp/lib/python3.12/site-packages/peft/tuners/adalora/config.py:96: UserWarning: Note that `r` is not used in AdaLora and will be ignored.If you intended to set the initial rank, use `init_r` instead.
  warnings.warn(
/tmp/ipykernel_16701/3814343219.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  return Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


[AdaLoRA] steps=1000, tinit=50, tfinal=300
Mode: adalora | Base: fp/bf16 (LoRA)
trainable params: 1,254,048 || all params: 224,157,768 || trainable%: 0.5594
Batch keys: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids']


Epoch,Training Loss,Validation Loss
1,1.701400,1.819320
2,1.627600,1.802031


Training done.

Trainable parameters: {'total': 224157768, 'trainable': 1254048, 'trainable_pct': 0.5594488253469762}
GPU memory: {'allocated_bytes': 485116416, 'max_allocated_bytes': 1075312640, 'reserved_bytes': 1088421888, 'max_reserved_bytes': 1231028224}
Training time (s): 444.88

Metrics (beam): {'rouge1': 0.2327980206927575, 'rouge2': 0.0643394655159361, 'rougeL': 0.1726707795128848, 'bleu': 0.05382795411296747}
Metrics (greedy): {'rouge1': 0.3382428051976542, 'rouge2': 0.12107612249654101, 'rougeL': 0.2699932019763703, 'bleu': 0.091354210737194}
Metrics (sample): {'rouge1': 0.20232671230314594, 'rouge2': 0.05703809851350835, 'rougeL': 0.15766975594155486, 'bleu': 0.03519468025439135}

Examples:
--------------------------------------------------------------------------------
INPUT: summarize: (CNN)Share, and your gift will be multiplied. That may sound like an esoteric adage, but when Zully Broussard selflessly decided to give one of her kidneys to a stranger, her generosity pai

In [10]:
CONFIG = {
    "model_name": "t5-base",    
    "sample_train": 2000,       
    "out_dir": "./out/t5_qlora",
    "epochs": 2,
    "train_bs": 4,
    "eval_bs": 4,
    "mode": "qlora",          
}

results = run_t5_peft(
    model_name=CONFIG["model_name"],
    sample_train=CONFIG["sample_train"],
    out_dir=CONFIG["out_dir"],
    epochs=CONFIG["epochs"],
    train_bs=CONFIG["train_bs"],
    eval_bs=CONFIG["eval_bs"],
    mode=CONFIG["mode"],
)
print("Training done.\n")

print("Trainable parameters:", results["trainable_params"])
print("GPU memory:", results["gpu_mem"]) 
print("Training time (s):", round(results["train_time_s"], 2))
print("\nMetrics (beam):", results["metrics"]["beam"]) 
print("Metrics (greedy):", results["metrics"]["greedy"]) 
print("Metrics (sample):", results["metrics"]["sample"]) 

print("\nExamples:")
for i, src in enumerate(results["examples"]["inputs"]):
    print("-" * 80)
    print("INPUT:", src[:200].replace("\n", " ") + ("..." if len(src) > 200 else ""))
    print("BEAM:", results["examples"]["beam"][i])
    print("GREEDY:", results["examples"]["greedy"][i])
    print("SAMPLE:", results["examples"]["sample"][i])


Mode: qlora | Base: 4-bit (QLoRA)
trainable params: 5,013,504 || all params: 227,917,056 || trainable%: 2.1997
Batch keys: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids']


/tmp/ipykernel_16701/3814343219.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  return Trainer(


Epoch,Training Loss,Validation Loss
1,1.661400,1.791404
2,1.572600,1.787696


Training done.

Trainable parameters: {'total': 157138176, 'trainable': 5013504, 'trainable_pct': 3.1905066786571328}
GPU memory: {'allocated_bytes': 479298560, 'max_allocated_bytes': 1426023424, 'reserved_bytes': 1545601024, 'max_reserved_bytes': 1545601024}
Training time (s): 427.91

Metrics (beam): {'rouge1': 0.25060399917542775, 'rouge2': 0.07116201957369696, 'rougeL': 0.18386312100597815, 'bleu': 0.06103131119773303}
Metrics (greedy): {'rouge1': 0.3072875759646112, 'rouge2': 0.10165843974740738, 'rougeL': 0.24671616893623644, 'bleu': 0.07539425731206427}
Metrics (sample): {'rouge1': 0.2827272727272727, 'rouge2': 0.07000107851596203, 'rougeL': 0.20396976408604311, 'bleu': 0.0375466694474461}

Examples:
--------------------------------------------------------------------------------
INPUT: summarize: (CNN)Share, and your gift will be multiplied. That may sound like an esoteric adage, but when Zully Broussard selflessly decided to give one of her kidneys to a stranger, her generosity

### Задание 2: Применение различных PEFT методов для задачи машинного перевода

**Цель задания**: Сравнить эффективность различных Parameter Efficient Fine-Tuning (PEFT) подходов для задачи автоматического перевода с английского на русский. В рамках задания необходимо реализовать и сравнить минимум 3 различных PEFT метода, проанализировать их влияние на качество перевода и эффективность обучения.

#### Задачи:

1. **Выбор датасета**:
   - Загрузите параллельный датасет для перевода, например, [Opus Books](https://huggingface.co/datasets/Helsinki-NLP/opus_books/viewer/en-ru), который содержит тексты на английском языке и их переводы на русский.
   - Используйте библиотеку `datasets` для загрузки и обработки данных. Убедитесь, что данные содержат параллельные тексты для обучения модели переводу.

2. **Предобработка данных**:
   - **Разделите данные** на обучающую и тестовую выборки (например, 80% для обучения и 20% для тестирования).
   - **Очистите текст**, удалив лишние пробелы и специальные символы, которые могут повлиять на обучение модели.
   - **Подготовьте данные** в формате, подходящем для выбранной модели:
     - Для GPT-2 данные должны быть в виде последовательности токенов, где исходный текст и перевод разделены специальными символами (например, `<|startoftext|>` для начала текста и `<|endoftext|>` для его конца).
     - Для T5 данные подаются в формате задачи перевода: входной текст начинается с задания `"translate English to Russian: <текст на английском>"`, а на выходе модель должна сгенерировать перевод.

3. **Создание модели**:
   - **GPT-2**:
     - Импортируйте предобученную модель `GPT2LMHeadModel` из библиотеки Hugging Face.
     - GPT-2 изначально не обучена для задачи перевода, поэтому нужно будет использовать специальную подготовку данных и дообучение на параллельных текстах.
     - Убедитесь, что модель настроена для генерации текста, ограничивая длину вывода для перевода.
   
   - **T5**:
     - Импортируйте модель `T5ForConditionalGeneration`, которая предобучена на множестве задач, включая перевод. T5 — это модель с условной генерацией, которая использует специальную задачу (`task`) для перевода.
     - Подготовьте модель для выполнения задачи перевода с английского на русский, используя предобученные веса и формат входных данных.

4. **Настройка параметров обучения**:
   - Настройте параметры обучения для обеих моделей:
     - Количество эпох (например, 3-5 эпох).
     - Размер батча (например, 16 или 32).
     - Скорость обучения (рекомендуется начать с 5e-5 и адаптировать в зависимости от потерь на валидации).
   - Используйте подходящие оптимизаторы, такие как AdamW, и функцию потерь для задачи перевода:
     - Для T5 подойдёт стандартная кросс-энтропийная функция потерь.
     - Для GPT-2 используйте ту же функцию с учётом последовательной генерации текста (автогрегрессии).

5. **Сравнительное исследование PEFT методов** (ключевая часть задания):
   - **Обязательно** реализуйте и сравните следующие PEFT подходы:
     - **LoRA**: Настройте различные значения rank (4, 8, 16, 32), alpha (16, 32, 64)
     - **QLoRA**: Комбинация 4-bit квантизации с LoRA для экономии памяти
     - **AdaLoRA**: Адаптивное изменение ранга с бюджетом параметров
     - **Дополнительные методы** (на выбор): IA³, Prefix Tuning, P-Tuning v2, или (IA)³
   - Для каждого PEFT метода проведите **grid search** по ключевым гиперпараметрам
   - **Baseline**: обязательно сравните с полным fine-tuning (если позволяют ресурсы)
   - Зафиксируйте для каждого эксперимента:
     - Процент обучаемых параметров от общего числа
     - Потребление GPU памяти (в GB)
     - Время обучения на эпоху
     - Скорость инференса (токенов/сек)
   - Используйте `SFTTrainer` или `Trainer` для стабильного процесса обучения

6. **Инференс**:
   - Используйте дообученную модель для перевода текстов с английского на русский на тестовой выборке.
   - Подготовьте несколько примеров для перевода и выведите результаты:
     - Для GPT-2 используйте автогрегрессивное декодирование текста.
     - Для T5 применяйте стандартные стратегии декодирования (например, greedy decoding или beam search).
   
7. **Комплексная оценка и анализ PEFT методов**:
   - **Автоматические метрики**:
     - **BLEU** (corpus-level и sentence-level) — основная метрика для машинного перевода
     - **chrF** — character-level F-score для более точной оценки морфологически богатых языков
     - **COMET** — нейронная метрика качества перевода (если позволяют ресурсы)
     - **ROUGE** — для дополнительной оценки похожести
   - **Создайте детальную сравнительную таблицу**:
     - Эффективность (% параметров, время, память) vs Качество (BLEU, chrF)
     - Pareto-frontier анализ: какие методы обеспечивают лучший trade-off
   - **Качественный анализ**: 
     - Проанализируйте примеры переводов от каждого PEFT метода
     - Определите типы ошибок, характерные для каждого подхода
     - Оцените стабильность качества на различных типах текстов

#### Ожидаемые результаты:
- **Исследовательский код** с реализацией всех протестированных PEFT методов
- **Научно-обоснованный отчет** с детальным сравнением эффективности PEFT подходов
- **Визуализации**: графики Pareto-frontier, сравнительные диаграммы по метрикам
- **Практические рекомендации**: когда использовать каждый PEFT метод в зависимости от ограничений
- **Воспроизводимые результаты**: четкие инструкции по повторению экспериментов

#### Рекомендуемые ресурсы:
- **[PEFT Documentation](https://huggingface.co/docs/peft/index)** - полная документация библиотеки PEFT
- **[LoRA Paper & Implementation](https://arxiv.org/abs/2106.09685)** - оригинальная статья LoRA
- **[QLoRA Paper](https://arxiv.org/abs/2305.14314)** - квантизованный LoRA подход
- **[AdaLoRA Paper](https://arxiv.org/abs/2303.10512)** - адаптивный LoRA
- **[PEFT Task Guides](https://huggingface.co/docs/peft/task_guides/translation)** - гайды по применению PEFT для разных задач
- [Документация Hugging Face Transformers](https://huggingface.co/docs/transformers/index)
- [Документация Hugging Face Datasets](https://huggingface.co/docs/datasets/index)
- **[BitsAndBytes](https://huggingface.co/docs/bitsandbytes/index)** - для квантизации в QLoRA
- **[TRL Library](https://huggingface.co/docs/trl/index)** - для эффективного обучения

#### Критерии оценки:
- **Корректность реализации PEFT методов** (35%) - правильная настройка минимум 3 различных PEFT подходов
- **Качество сравнительного анализа** (25%) - детальное сравнение по всем метрикам эффективности и качества
- **Научная обоснованность выводов** (20%) - аргументированные рекомендации по выбору методов
- **Воспроизводимость результатов** (10%) - четкие инструкции и фиксированные семена
- **Качество перевода** (10%) - достижение конкурентоспособных результатов по BLEU/chrF метрикам

In [1]:

# %%
from __future__ import annotations
import re, random
from dataclasses import dataclass, asdict
from typing import Dict, List, Tuple

import torch
from torch import cuda

from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split

from transformers import (
    GPT2LMHeadModel, GPT2Tokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    Trainer, TrainingArguments
)

# PEFT
from peft import (
    get_peft_model,
    LoraConfig,
    AdaLoraConfig,
    TaskType,
    prepare_model_for_kbit_training,
)

# Metrics
import evaluate
import sacrebleu
from sacrebleu.metrics import CHRF


@dataclass
class Config:
    # Data
    dataset_name: str = "Helsinki-NLP/opus_books"
    dataset_subset: str = "en-ru"
    test_size: float = 0.2
    seed: int = 42
    # Model / Tokenizer
    base_model: str = "gpt2"
    max_length: int = 512
    # LoRA / AdaLoRA hyperparams
    lora_r: int = 8
    lora_alpha: int = 32
    lora_dropout: float = 0.1
    adalora_target_r: int = 8
    adalora_init_r: int = 12
    adalora_tinit: int = 200
    adalora_tfinal: int = 1000
    adalora_deltaT: int = 10
    # QLoRA (bnb) config
    qlora_4bit: bool = True
    qlora_compute_dtype: torch.dtype = torch.bfloat16
    # Training
    output_dir_base: str = "./gpt2_peft_mt"
    epochs: int = 5
    train_batch_size: int = 10
    eval_batch_size: int = 10
    grad_accum_steps: int = 8
    lr: float = 5e-5
    fp16: bool = False
    logging_steps: int = 50
    save_total_limit: int = 2
    eval_steps: int = 500
    # Generation for eval
    gen_max_new_tokens: int = 128
    gen_num_beams: int = 5
    # Eval subset size
    eval_samples: int = 500

CFG = Config()
random.seed(CFG.seed)
DEVICE = "cuda" if cuda.is_available() else "cpu"
print("Using device:", DEVICE)


/home/vladislav/miniconda3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:

# %% [markdown]
# ### 2) Data loading & preparation helpers

# %%
def clean_text(s: str) -> str:
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    return s

def load_and_prepare_data(cfg: Config) -> Tuple[DatasetDict, Dataset, Dataset]:
    raw = load_dataset(cfg.dataset_name, cfg.dataset_subset)
    rows = [
        {"en": clean_text(x["translation"]["en"]), "ru": clean_text(x["translation"]["ru"])}
        for x in raw["train"]
        if x["translation"]["en"] is not None and x["translation"]["ru"] is not None
    ]
    train_rows, test_rows = train_test_split(rows, test_size=cfg.test_size, random_state=cfg.seed)
    train_ds = Dataset.from_list(train_rows)
    test_ds = Dataset.from_list(test_rows)
    ds = DatasetDict({"train": train_ds, "test": test_ds})
    return ds, train_ds, test_ds

def build_prompt(example: Dict[str, str]) -> str:
    # Causal-LM as Seq2Seq via tags
    return f"<|startoftext|> {example['en']} <|sep|> {example['ru']} <|endoftext|>"

def to_causal_format(ds: Dataset) -> Dataset:
    return ds.map(lambda ex: {"text": build_prompt(ex)})


In [3]:

# %% [markdown]
# ### 3) Tokenizer & Base Model (no PEFT applied yet)
# This cell only creates the tokenizer and a **base** GPT-2 model.
# PEFT adapters will be attached in the dedicated LoRA/AdaLoRA/QLoRA cells.

# %%
def load_tokenizer(cfg: Config) -> GPT2Tokenizer:
    tok = GPT2Tokenizer.from_pretrained(cfg.base_model)
    special_tokens = {
        "pad_token": "<|pad|>",
        "bos_token": "<|startoftext|>",
        "eos_token": "<|endoftext|>",
        "additional_special_tokens": ["<|sep|>"],
    }
    tok.add_special_tokens(special_tokens)
    return tok

def load_base_model(cfg: Config, tokenizer: GPT2Tokenizer, quantization_config=None) -> GPT2LMHeadModel:
    model = GPT2LMHeadModel.from_pretrained(
        cfg.base_model,
        device_map="auto" if quantization_config is not None else None,
        quantization_config=quantization_config,
    )
    model.resize_token_embeddings(len(tokenizer))
    return model


In [4]:

# %% [markdown]
# ### 4) Tokenization, Generation, and Metrics
# Common utilities used by all three methods.

# %%
def tokenize_fn(tokenizer: GPT2Tokenizer, max_length: int):
    def _tok(batch):
        return tokenizer(batch["text"], truncation=True, max_length=max_length)
    return _tok

def generate_translation(model: GPT2LMHeadModel, tokenizer: GPT2Tokenizer, src_en: str, cfg: Config) -> str:
    model.eval()
    prompt = f"<|startoftext|> {src_en} <|sep|>"
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=cfg.gen_max_new_tokens,
            num_beams=cfg.gen_num_beams,
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id,
        )
    text = tokenizer.decode(out_ids[0], skip_special_tokens=False)
    if "<|sep|>" in text:
        text = text.split("<|sep|>", 1)[1]
    return text.replace("<|endoftext|>", "").replace("<|pad|>", "").strip()

def compute_metrics_on_subset(model, tokenizer, test_raw: Dataset, cfg: Config) -> Dict[str, float]:
    n = min(len(test_raw), cfg.eval_samples)
    sample = test_raw.shuffle(seed=CFG.seed).select(range(n))
    preds, refs = [], []
    for ex in sample:
        src, ref = ex["en"], ex["ru"]
        hyp = generate_translation(model, tokenizer, src, cfg)
        preds.append(hyp)
        refs.append(ref)

    bleu = sacrebleu.corpus_bleu(preds, [refs]).score
    chrf = CHRF().corpus_score(preds, [refs]).score
    rouge_metric = evaluate.load("rouge")
    rouge_res = rouge_metric.compute(predictions=preds, references=refs, use_stemmer=True)
    return {
        "BLEU": bleu,
        "chrF": chrf,
        "ROUGE-1": rouge_res.get("rouge1", 0.0) * 100,
        "ROUGE-2": rouge_res.get("rouge2", 0.0) * 100,
        "ROUGE-L": rouge_res.get("rougeL", 0.0) * 100,
        "ROUGE-Lsum": rouge_res.get("rougeLsum", 0.0) * 100,
    }

def build_trainer(model, tokenizer, train_tok, test_tok, cfg: Config, out_dir_suffix: str) -> Trainer:
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    args = TrainingArguments(
        output_dir=f"{cfg.output_dir_base}/{out_dir_suffix}",
        per_device_train_batch_size=cfg.train_batch_size,
        per_device_eval_batch_size=cfg.eval_batch_size,
        gradient_accumulation_steps=cfg.grad_accum_steps,
        num_train_epochs=cfg.epochs,
        learning_rate=cfg.lr,
        fp16=cfg.fp16,
        logging_steps=cfg.logging_steps,
        eval_strategy="steps",
        save_strategy="steps",
        eval_steps=cfg.eval_steps,
        save_steps=cfg.eval_steps,
        save_total_limit=cfg.save_total_limit,
        report_to="none",
        load_best_model_at_end=False,
    )
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_tok,
        eval_dataset=test_tok,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    return trainer


In [5]:

# %% [markdown]
# ### 5) Prepare Dataset Once (used by all methods)
# Run this once; it caches tokenized data for reuse.

# %%
print("Config:", asdict(CFG))
ds, train_raw, test_raw = load_and_prepare_data(CFG)
print(ds)

train_fmt = to_causal_format(train_raw)
test_fmt  = to_causal_format(test_raw)
print("Sample formatted:", train_fmt[0]["text"][:200])

tokenizer = load_tokenizer(CFG)

tok_fn = tokenize_fn(tokenizer, CFG.max_length)
train_tok = train_fmt.map(tok_fn, batched=True, remove_columns=train_fmt.column_names)
test_tok  = test_fmt.map(tok_fn, batched=True, remove_columns=test_fmt.column_names)

print("Train tokenized columns:", train_tok.column_names)


Config: {'dataset_name': 'Helsinki-NLP/opus_books', 'dataset_subset': 'en-ru', 'test_size': 0.2, 'seed': 42, 'base_model': 'gpt2', 'max_length': 512, 'lora_r': 8, 'lora_alpha': 32, 'lora_dropout': 0.1, 'adalora_target_r': 8, 'adalora_init_r': 12, 'adalora_tinit': 200, 'adalora_tfinal': 1000, 'adalora_deltaT': 10, 'qlora_4bit': True, 'qlora_compute_dtype': torch.bfloat16, 'output_dir_base': './gpt2_peft_mt', 'epochs': 5, 'train_batch_size': 10, 'eval_batch_size': 10, 'grad_accum_steps': 8, 'lr': 5e-05, 'fp16': False, 'logging_steps': 50, 'save_total_limit': 2, 'eval_steps': 500, 'gen_max_new_tokens': 128, 'gen_num_beams': 5, 'eval_samples': 500}
DatasetDict({
    train: Dataset({
        features: ['en', 'ru'],
        num_rows: 13996
    })
    test: Dataset({
        features: ['en', 'ru'],
        num_rows: 3500
    })
})


Map: 100%|██████████| 3500/3500 [00:00<00:00, 37433.77 examples/s]


Sample formatted: <|startoftext|> The rye – after he had so long held out for a certain price – was sold fifty kopeks a chetvert cheaper than had been offered him a month ago. <|sep|> Рожь, цену на которую он так долго


Map: 100%|██████████| 3500/3500 [00:01<00:00, 1884.40 examples/s]

Train tokenized columns: ['input_ids', 'attention_mask']


In [6]:
# %%
def prepare_for_training(model, tokenizer=None):
    # Disable KV cache when using gradient checkpointing
    if hasattr(model.config, "use_cache"):
        model.config.use_cache = False

    # Ensure inputs can require grads (important when most weights are frozen with PEFT)
    if hasattr(model, "enable_input_require_grads"):
        model.enable_input_require_grads()
    else:
        model.get_input_embeddings().requires_grad_(True)

    # Be explicit about PAD token id (prevents some generate() mishaps)
    if tokenizer is not None and getattr(model.config, "pad_token_id", None) is None:
        model.config.pad_token_id = tokenizer.pad_token_id

    # Sanity check: confirm we actually have trainable params
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"Trainable params: {trainable:,} / {total:,} ({100*trainable/total:.4f}% trainable)")
    return model


In [7]:

# %% [markdown]
# ## 6) LoRA — train & evaluate (run this cell to do plain LoRA)
# This cell attaches **LoRA adapters**, trains, prints a few samples, and reports **BLEU/chrF/ROUGE**.

# %%
# --- Base model (no quantization here) ---
model_lora = load_base_model(CFG, tokenizer)

# --- Attach LoRA ---
lora_cfg = LoraConfig(
    r=CFG.lora_r,
    lora_alpha=CFG.lora_alpha,
    target_modules=["c_attn"],  # GPT-2 attention proj
    lora_dropout=CFG.lora_dropout,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
model_lora = get_peft_model(model_lora, lora_cfg)
model_lora.gradient_checkpointing_enable()
model_lora = prepare_for_training(model_lora, tokenizer)   # <-- add this line
model_lora.to(DEVICE)

# --- Train ---
trainer_lora = build_trainer(model_lora, tokenizer, train_tok, test_tok, CFG, out_dir_suffix="lora")
trainer_lora.train()

# --- Quick qualitative check ---
for i in range(3):
    ex = test_raw[i]
    hyp = generate_translation(model_lora, tokenizer, ex["en"], CFG)
    print("-"*60)
    print("EN:", ex["en"])
    print("REF:", ex["ru"])
    print("HYP:", hyp)

# --- Metrics ---
print("\n[LoRA] Computing metrics...")
scores = compute_metrics_on_subset(model_lora, tokenizer, test_raw, CFG)
for k, v in scores.items():
    print(f"{k:10s}: {v:.2f}")


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
/home/vladislav/miniconda3/envs/nlp/lib/python3.12/site-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Trainable params: 294,912 / 124,737,024 (0.2364% trainable)


/tmp/ipykernel_29199/2514287141.py:70: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 50258, 'pad_token_id': 50257}.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
500,2.559000,2.480579
1000,2.483200,2.401433
1500,2.456800,2.363074
2000,2.437500,2.341262
2500,2.408900,2.326287
3000,2.416500,2.313534
3500,2.406900,2.307347
4000,2.405300,2.302206


/home/vladislav/miniconda3/envs/nlp/lib/python3.12/site-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/home/vladislav/miniconda3/envs/nlp/lib/python3.12/site-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/home/vladislav/miniconda3/envs/nlp/lib/python3.12/site-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/home/vladislav/miniconda3/envs/nlp/lib/python3.12/site-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/home/vladislav/miniconda3/envs/nlp/lib/python3.12/site-packages/peft/utils/save_and

------------------------------------------------------------
EN: But suddenly she heard the rustle of a dress and a burst of suppressed sobbing. A pair of arms encircled her neck from below and Kitty was kneeling before her.
REF: Но вдруг она услыхала шум платья и вместе звук разразившегося сдержанного рыданья, и чьи-то руки снизу обняли ее шею. Кити на коленях стояла пред ней.
HYP: Она она она она она она она она она она она она она она она.
------------------------------------------------------------
EN: 'Yes, tell me what is happening in Pokrovsk Is the house still standing, and the birch trees, and our schoolroom?
REF: -- Да расскажи мне, что делается в Покровском? Что, дом все стоит, и березы, и наша классная?
HYP: -- Она, что что что что что что что что что что что что что что что что что что что что что что что что �
------------------------------------------------------------
EN: 'Matthew!' he called, 'will you and Mary arrange everything for Anna Arkadyevna in the little sitti

In [9]:

# %% [markdown]
# ## 8) QLoRA — train & evaluate (run this cell for QLoRA)
# 4-bit quantization with bitsandbytes.  
# **Requires** a GPU + matching CUDA build of `torch` and `bitsandbytes`.

# %%
# --- BitsAndBytes quantization config ---
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=CFG.qlora_4bit,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=CFG.qlora_compute_dtype,
)

# --- Base model in 4-bit ---
model_qlora = load_base_model(CFG, tokenizer, quantization_config=bnb_cfg)

# --- Prepare & attach LoRA adapters for QLoRA ---
model_qlora = prepare_model_for_kbit_training(model_qlora)
qlora_lora_cfg = LoraConfig(
    r=CFG.lora_r,
    lora_alpha=CFG.lora_alpha,
    target_modules=["c_attn"],
    lora_dropout=CFG.lora_dropout,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
model_qlora = get_peft_model(model_qlora, qlora_lora_cfg)
model_qlora.gradient_checkpointing_enable()
# device_map="auto" is set already; no explicit .to(DEVICE)

# --- Train ---
trainer_qlora = build_trainer(model_qlora, tokenizer, train_tok, test_tok, CFG, out_dir_suffix="qlora")
trainer_qlora.train()

# --- Quick qualitative check ---
for i in range(3):
    ex = test_raw[i]
    hyp = generate_translation(model_qlora, tokenizer, ex["en"], CFG)
    print("-"*60)
    print("EN:", ex["en"])
    print("REF:", ex["ru"])
    print("HYP:", hyp)

# --- Metrics ---
print("\n[QLoRA] Computing metrics...")
scores = compute_metrics_on_subset(model_qlora, tokenizer, test_raw, CFG)
for k, v in scores.items():
    print(f"{k:10s}: {v:.2f}")



/tmp/ipykernel_39793/2514287141.py:70: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 50258, 'pad_token_id': 50257}.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
500,2.586400,2.459620


/home/vladislav/miniconda3/envs/nlp/lib/python3.12/site-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/home/vladislav/miniconda3/envs/nlp/lib/python3.12/site-packages/peft/utils/save_and_load.py:300: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


------------------------------------------------------------
EN: But suddenly she heard the rustle of a dress and a burst of suppressed sobbing. A pair of arms encircled her neck from below and Kitty was kneeling before her.
REF: Но вдруг она услыхала шум платья и вместе звук разразившегося сдержанного рыданья, и чьи-то руки снизу обняли ее шею. Кити на коленях стояла пред ней.
HYP: Привать не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не не
------------------------------------------------------------
EN: 'Yes, tell me what is happening in Pokrovsk Is the house still standing, and the birch trees, and our schoolroom?
REF: -- Да расскажи мне, что делается в Покровском? Что, дом все стоит, и березы, и наша классная?
HYP: -- Она прошевал прошевал прошевал прошевал прошевал прошевал прошевал прошевал.
------------------------------------------------------------
EN: 'Matthew!' he called, 'will you and Mary arrange everythin